In [ ]:
!pip install ultralytics
!pip install pytorch-lightning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/abinaya/Earthcam/Facial_Recognition/TempleFaceRecognition/face')

In [ ]:
import os
import shutil
import numpy as np
import cv2
import statistics
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from numpy import expand_dims
from collections import deque, Counter
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats as st
import torch
from PIL import Image
from torchvision import transforms
from ultralytics import YOLO
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics.pairwise import euclidean_distances
import net_1 as net

In [ ]:
# Load YOLO model
yolov8_model = YOLO(r"/content/drive/MyDrive/abinaya/Earthcam/Facial_Recognition/TempleFaceRecognition/face/yolov8m_200e.pt")
yolov8_model = yolov8_model.to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# AdaFace model
adaface_models = {
    'ir_50': "/content/drive/MyDrive/Sourav/TempleFaceRecognition/temple_20jun.ckpt",
}

def load_pretrained_model(architecture='ir_50'):
    assert architecture in adaface_models.keys()
    model = net.build_model(architecture)
    statedict = torch.load(adaface_models[architecture], map_location= torch.device('cpu'))['state_dict']
    model_statedict = {key[6:]: val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model

model = load_pretrained_model('ir_50')

def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    bgr_img = ((np_img[:, :, ::-1] / 255.) - 0.5) / 0.5
    tensor = torch.tensor([bgr_img.transpose(2, 0, 1)]).float()
    return tensor

In [ ]:
# Drawing functions
def draw_text(img, text, pos=(0, 0), font=cv2.FONT_HERSHEY_PLAIN, font_scale=3, text_color=(0, 255, 0), font_thickness=2, text_color_bg=(0, 0, 0)):
    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, (x, y - text_h - 20), (x + text_w + 20, y), text_color_bg, -1)
    cv2.putText(img, text, (x + 10, y - 10), font, font_scale, text_color, font_thickness)

def draw_bb_text(frame, text, bbox, font=cv2.FONT_HERSHEY_PLAIN, font_scale=3, text_color=(0, 255, 0), font_thickness=2, text_color_bg=(255, 255, 255), tboxh=14):
    startX, startY, endX, endY = bbox
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    startY = tboxh if startY < tboxh else startY
    startX = 1 if startX < 1 else startX
    bg = np.ones_like(frame[startY - tboxh:startY, startX - 1:startX + text_w + 3]).astype('uint8') * 255
    bg[:, :] = text_color_bg
    frame[startY - tboxh:startY, startX - 1:startX + text_w + 3] = cv2.addWeighted(frame[startY - tboxh:startY, startX - 1:startX + text_w + 3], 0.0, bg, 1.0, 1)
    cv2.putText(frame, text, (startX, startY - text_h + 2), font, font_scale, text_color, font_thickness)

def get_color_from_id(idx):
    idx = idx * 3
    color = (int((37 * idx) % 255), int((17 * idx) % 255), int((29 * idx) % 255))
    return color

def isLightOrDark(rgbColor=[0, 128, 255]):
    [r, g, b] = rgbColor
    hsp = np.sqrt(0.299 * (r * r) + 0.587 * (g * g) + 0.114 * (b * b))
    return [0, 0, 0] if hsp > 127.5 else [255, 255, 255]

In [ ]:
def format_detection_results(tracking_results, target_classes=None):
    formatted_results = []
    for tracking_result in tracking_results:
        bboxes = tracking_result.boxes.cpu().numpy()
        for xyxy, class_id in zip(bboxes.xyxy, bboxes.cls):
            if target_classes is None or class_id in target_classes:
                formatted_results.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), int(class_id)])
    return formatted_results

def format_tracking_results(tracking_results, target_classes=None):
    formatted_results = []
    for tracking_result in tracking_results:
        bboxes = tracking_result.boxes.cpu().numpy()
        for xyxy, tid, class_id in zip(bboxes.xyxy, bboxes.id, bboxes.cls):
            if target_classes is None or class_id in target_classes:
                formatted_results.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), int(tid), int(class_id)])
    return formatted_results

In [ ]:
def add_padding(pil_img, top, right, bottom, left, color=(0, 0, 0)):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

def get_aligned_face(image, yolo_model):
    results = yolo_model(image, verbose=False, stream=True)
    faces = format_detection_results(results)
    if len(faces) > 0:
        xmin, ymin, xmax, ymax = faces[0][:4]
        face = image[ymin:ymax, xmin:xmax]
        return face
    else:
        return None

def fx_adaface(img):
    img = cv2.resize(img, (112, 112))
    converted = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(converted)
    bgr_input = to_input(pil_image)
    features, _ = model(bgr_input)
    X = features.detach().numpy()
    return X

In [ ]:
gallery = r"/content/drive/MyDrive/abinaya/Earthcam/Facial_Recognition/TempleFaceRecognition/people_gallery/"
people_names = os.listdir(gallery)
people_images = {}

for name in people_names:
    people_images[name] = {}
    person_path = os.path.join(gallery, name)
    people_images[name]['image_paths'] = []
    people_images[name]['embeddings'] = []
    for file_name in os.listdir(person_path):
        file_path = os.path.join(person_path, file_name)
        people_images[name]['image_paths'].append(file_path)
        image = cv2.imread(file_path)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        face_img = get_aligned_face(rgb, yolov8_model)
        if face_img is not None:
            face_embedding = fx_adaface(face_img)
            people_images[name]['embeddings'].append(face_embedding)
    people_images[name]['embeddings'] = np.array(people_images[name]['embeddings']).reshape(-1, 512)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
<ipython-input-5-959803effcea>:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  tensor = torch.tensor([bgr_img.transpose(2, 0, 1)]).float()


In [ ]:
import cv2
import numpy as np

# Function to process a single frame
def process_frame(frame):
    rgb_planes = cv2.split(frame)

    result_planes = []
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)

    result = cv2.merge(result_planes)
    result_norm = cv2.merge(result_norm_planes)
    return result, result_norm

# Read the input video
input_video = cv2.VideoCapture('/content/drive/MyDrive/Sourav/vid2.mp4')

# Get the video's properties
frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(input_video.get(cv2.CAP_PROP_FPS))
frame_count = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the codec and create VideoWriter objects for the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('output_video.mp4', fourcc, fps, (frame_width, frame_height))
output_video_norm = cv2.VideoWriter('output_video_norm.mp4', fourcc, fps, (frame_width, frame_height))

frame_number = 0
while input_video.isOpened():
    ret, frame = input_video.read()
    if not ret:
        break

    # Process the frame
    result, result_norm = process_frame(frame)

    # Write the processed frame to the output video
    output_video.write(result)
    output_video_norm.write(result_norm)

    frame_number += 1
    print(f'Processed frame {frame_number}/{frame_count}')

# Release everything if job is finished
input_video.release()
output_video.release()
output_video_norm.release()
cv2.destroyAllWindows()


In [ ]:
#@title Dot Product

# Define the input and output video paths
input_video_path = r"/content/drive/MyDrive/Sourav/vid2-online-video-cuttercom_lK2OmAF5.mp4"
output_video_path = r"content/dotproduct.mp4"

# Open the input video
input_video = cv2.VideoCapture(input_video_path)

# Get the width and height of the input video
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

# Buffer to store recognized names for smoothing
buffer_size = 20

face_buffer = {}
# Process each frame of the input video

n_unknown = 0
unknown_people = []

while True:
    ret, frame = input_video.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = yolov8_model.track(rgb, persist=True, tracker="bytetrack.yaml", verbose=False, stream=True)
    try:
        faces = format_tracking_results(res)
    except:
        faces = []

    known_people = []
    for face in faces:

        xmin, ymin, xmax, ymax, face_id = face[:5]
        face_img = frame[ymin:ymax, xmin:xmax]
        face_embedding = fx_adaface(face_img)

        scores = []
        for name in people_names:
            max_score = np.dot(face_embedding, people_images[name]['embeddings'].T).max()
            scores.append(max_score)

        scores = np.array(scores)

        if scores.max() < 0.2:
            score = round(scores.max(), 2)
            tid = 0
            name = 'Unknown'
        else:
            score = round(scores.max(), 2)
            tid = scores.argmax() + 1
            name = people_names[scores.argmax()]
            if name == "Unknown":
                tid = 0

        print(f"name : {name}")

        try:
            face_buffer[face_id].append(name)
            if len(face_buffer[face_id]) > buffer_size:
                face_buffer[face_id].pop(0)

            names_with_counts = np.unique(face_buffer[face_id], return_counts=True)
            name = names_with_counts[0][np.argmax(names_with_counts[1])]
            if name == "Unknown":
                tid = 0
            else:
                tid = people_names.index(name) + 1

        except:
            face_buffer[face_id] = [name]

        if name == "Unknown":
            n_unknown += 1
            unknown_people.append(face_id)
            unknown_people = np.unique(unknown_people).tolist()
        if name != "Unknown":
            known_people.append(name)

        startX, startY, endX, endY = xmin, ymin, xmax, ymax


        bg_color = get_color_from_id(tid)
        text_color = isLightOrDark(bg_color)

        cv2.rectangle(frame, (startX, startY), (endX, endY), bg_color, 1)
        draw_bb_text(frame,f" {name} ", (startX, startY, endX, endY),cv2.FONT_HERSHEY_DUPLEX, 0.3, text_color, 1, bg_color, tboxh=10)


    output_video.write(frame)

# Release the video objects
input_video.release()
output_video.release()

print("\n\n\nOutput video saved as './new_video_1.mp4'")


name : Akash
name : Bhavna
name : Prashant
name : Bhavna
name : Dil
name : Bhavna
name : Bhavna
name : Aarav
name : Diya
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhavna
name : Bhav

In [ ]:
known_people


In [ ]:
face_buffer


{1: ['Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Bhavna',
  'Gopal',
  'Vikram',
  'Prashant'],
 2: ['Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prakash',
  'Prashant',
  'Shubham',
  'Prakash',
  'Prakash',
  'Vikram'],
 3: ['Naveen',
  'Naveen',
  'Naveen',
  'Naveen',
  'Nitish',
  'Gaurav',
  'Aarav',
  'Gaurav',
  'Shubham',
  'Naveen',
  'Naveen',
  'Arnab',
  'Aarav',
  'Aarav',
  'Aarav',
  'Aarav',
  'Aarav',
  'Aarav',
  'Aarav',
  'Vikram'],
 4: ['Diya',
  'Diya',
  'Diya',
  'Diya',
  'Diya',
  'Diya',
  'Diya',
  'Diya',
  'Bhavna',
  'Diya',
  'Diya',
  'Unknown',
  'Diya',
  'Diya',
  'Diya',
  'Unknown',
  'Unknown',
  'Aarav',
  'Arjun',
  'Prakash'],
 5: ['Aarav',
  'Aarav',
  'Aarav',

In [ ]:
total_face

In [ ]:
#@title Dot Product

# Define the input and output video paths
input_video_path = r"/content/drive/MyDrive/Sourav/vid2.avi"
output_video_path = './dotproduct.mp4'

# Open the input video
input_video = cv2.VideoCapture(input_video_path)

# Get the width and height of the input video
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

# Buffer to store recognized names for smoothing
buffer_size = 20

face_buffer = {}

# Process each frame of the input video
n_unknown = 0
unknown_people = []

while True:
    ret, frame = input_video.read()
if not ret:
    break

scss
Copy code
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
res = yolov8_model.track(rgb, persist=True, tracker="bytetrack.yaml", verbose=False, stream=True)
try:
    faces = format_tracking_results(res)
except:
    faces = []

known_people = []
for face in faces:

    xmin, ymin, xmax, ymax, face_id = face[:5]
    face_img = frame[ymin:ymax, xmin:xmax]
    face_embedding = fx_adaface(face_img)

    scores = []
    for name in people_names:
        max_score = np.dot(face_embedding, people_images[name]['embeddings'].T).max()
        scores.append(max_score)

    scores = np.array(scores)

    if scores.max() < 0.2:
        score = round(scores.max(), 2)
        tid = 0
        name = 'Unknown'
    else:
        score = round(scores.max(), 2)
        tid = scores.argmax() + 1
        name = people_names[scores.argmax()]
        if name == "Unknown":
            tid = 0

    print(f"name : {name}")

    try:
        face_buffer[face_id].append(name)
        names_with_counts = np.unique(face_buffer[face_id], return_counts=True)
        name = names_with_counts[0][np.argmax(names_with_counts[1])]
        if name == "Unknown":
            tid = 0
        else:
            tid = people_names.index(name) + 1

    except:
        face_buffer[face_id] = [name]

    if name == "Unknown":
        n_unknown += 1
        unknown_people.append(face_id)
        unknown_people = np.unique(unknown_people).tolist()
    if name != "Unknown":
        known_people.append(name)

    startX, startY, endX, endY = xmin, ymin, xmax, ymax


    bg_color = get_color_from_id(tid)
    text_color = isLightOrDark(bg_color)

    cv2.rectangle(frame, (startX, startY), (endX, endY), bg_color, 1)
    draw_bb_text(frame,f" {name} ", (startX, startY, endX, endY),cv2.FONT_HERSHEY_DUPLEX, 0.3, text_color, 1, bg_color, tboxh=10)


output_video.write(frame)
Release the video objects
input_video.release()
output_video.release()

print("\n\n\nOutput video saved as './new_video_1.mp4'")

In [ ]:
#@title Euclidean

# Define the input and output video paths
input_video_path = r"/content/new_video_resized.mp4"
output_video_path = './euclidean.mp4'

# Open the input video
input_video = cv2.VideoCapture(input_video_path)

# Get the width and height of the input video
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

# Buffer to store recognized names for smoothing
buffer_size = 20

face_buffer = {}
# Process each frame of the input video

n_unknown = 0
unknown_people = []

while True:
    ret, frame = input_video.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = yolov8_model.track(rgb, persist=True, tracker="bytetrack.yaml", verbose=False, stream=True)
    try:
        faces = format_tracking_results(res)
    except:
        faces = []

    known_people = []
    for face in faces:

        xmin, ymin, xmax, ymax, face_id = face[:5]
        face_img = frame[ymin:ymax, xmin:xmax]
        face_embedding = fx_adaface(face_img)

        dist = []
        for name in people_names:
            dist.append(euclidean_distances(face_embedding.reshape(1, -1), people_images[name]['embeddings']).min())

        dist = np.array(dist)

        if dist.min() > 1.3:
            score = round(dist.min(), 2)
            tid = 0
            name = 'Unknown'
        else:
            score = round(dist.min(), 2)
            tid = dist.argmin() + 1
            name = people_names[dist.argmin()]
            if name == "Unknown":
                tid = 0

        print(f"name : {name}")

        try:
            face_buffer[face_id].append(name)
            names_with_counts = np.unique(face_buffer[face_id], return_counts=True)
            name = names_with_counts[0][np.argmax(names_with_counts[1])]
            if name == "Unknown":
                tid = 0
            else:
                tid = people_names.index(name) + 1

        except:
            face_buffer[face_id] = [name]

        if name == "Unknown":
            n_unknown += 1
            unknown_people.append(face_id)
            unknown_people = np.unique(unknown_people).tolist()
        if name != "Unknown":
            known_people.append(name)

        startX, startY, endX, endY = xmin, ymin, xmax, ymax


        bg_color = get_color_from_id(tid)
        text_color = isLightOrDark(bg_color)

        cv2.rectangle(frame, (startX, startY), (endX, endY), bg_color, 1)
        draw_bb_text(frame,f" {name} ", (startX, startY, endX, endY),cv2.FONT_HERSHEY_DUPLEX, 0.3, text_color, 1, bg_color, tboxh=10)


    output_video.write(frame)

# Release the video objects
input_video.release()
output_video.release()

print("\n\n\nOutput video saved as './new_video_1.mp4'")




Output video saved as './new_video_1.mp4'


In [ ]:
#@title Cosine Similarity

# Define the input and output video paths
input_video_path = r"/content/new_video_resized.mp4"
output_video_path = './cosine.mp4'

# Open the input video
input_video = cv2.VideoCapture(input_video_path)

# Get the width and height of the input video
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

# Buffer to store recognized names for smoothing
buffer_size = 20

face_buffer = {}
# Process each frame of the input video

n_unknown = 0
unknown_people = []

while True:
    ret, frame = input_video.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = yolov8_model.track(rgb, persist=True, tracker="bytetrack.yaml", verbose=False, stream=True)
    try:
        faces = format_tracking_results(res)
    except:
        faces = []

    known_people = []
    for face in faces:

        xmin, ymin, xmax, ymax, face_id = face[:5]

        face_img = frame[ymin:ymax, xmin:xmax]
        face_img = cv2.resize(face_img, (112, 112))
        face_img = Image.fromarray(face_img)
        face_tensor = to_input(face_img)
        face_embedding, _ = model(face_tensor)

        cos_sim = []
        for name in people_names:
            cos_sim.append(cosine_similarity(face_embedding.detach().numpy().reshape(1, -1), people_images[name]['embeddings']).max())

        cos_sim = np.array(cos_sim)

        if cos_sim.max() < 0.2:
            score = round(cos_sim.max(), 2)
            tid = 0
            name = 'Unknown'
        else:
            score = round(cos_sim.max(), 2)
            tid = cos_sim.argmax() + 1
            name = people_names[cos_sim.argmax()]
            if name == "Unknown":
                tid = 0

        print(f"name : {name}")

        try:
            face_buffer[face_id].append(name)
            names_with_counts = np.unique(face_buffer[face_id], return_counts=True)
            name = names_with_counts[0][np.argmax(names_with_counts[1])]
            if name == "Unknown":
                tid = 0
            else:
                tid = people_names.index(name) + 1

        except:
            face_buffer[face_id] = [name]

        if name == "Unknown":
            n_unknown += 1
            unknown_people.append(face_id)
            unknown_people = np.unique(unknown_people).tolist()
        if name != "Unknown":
            known_people.append(name)

        startX, startY, endX, endY = xmin, ymin, xmax, ymax

        bg_color = get_color_from_id(tid)
        text_color = isLightOrDark(bg_color)

        cv2.rectangle(frame, (startX, startY), (endX, endY), bg_color, 1)  # Reduced thickness
        draw_bb_text(frame,f" {name} ", (startX, startY, endX, endY),cv2.FONT_HERSHEY_DUPLEX, 0.3, text_color, 1, bg_color, tboxh=10)  # Reduced text size and background box size

    output_video.write(frame)

# Release the video objects
input_video.release()
output_video.release()

print("\n\n\nOutput video saved as './new_video_1.mp4'")

name : Manish
name : Prashant
name : Vikram
name : Vikram
name : Arjun
name : Manish
name : Arjun
name : Manish
name : Aarav
name : Vikram
name : Arjun
name : Vikram
name : Vikram
name : Aarav
name : Arjun
name : Arjun
name : Vikram
name : Arjun
name : Arjun
name : Vikram
name : Arjun
name : Vikram
name : Arjun
name : Vikram
name : Dev
name : Aarav
name : Vikram
name : Aarav
name : Vikram
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Vikram
name : Aarav
name : Aarav
name : Aarav
name : Aarav
name : Dev
name : Aarav
name : Vikram
name : Prakash
name : Vikram
name : Aarav
name : Vikram
name : Vikram
name : Dev
name : Vikram
name : Dev
name : Vikram
name : Vikram
name : Vikram
name : Vikram
name : Dev
name : Vikram
name : Aarav
name : Vikram
name : Gopal
name : Aarav
name